# Sawyer Jacobson
## Data Science Capstone, Midterm Project

Planned things to do:

- Data Preprocessing
    - Fill in missing values
    - Clean the 'target' salary data
    - Split into train and test data
- EDA
    - Plot distributions of variables
        - Games played is pretty big
    - Check for high correlations
    - Potentially look for outliers
    - Separate by position
- Modeling
    - Unsupervised clustering as feature engineering
        - Kmeans separately on forwards and defensemen
        - Used to potentially categorize offensive and defensive players in each group since that is where the clusters will most likely align
    - Potential feature selection since there are a fair amount of features for the number of observations
    - Supervised learning: predicting salary
        - Use and optimize a few different models with RMSE as the error metric


In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import plotnine
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import scipy
from sklearn import metrics
import sklearn as sk
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Data Preprocessing

Here we will read in the data and explore, fill in missing values, and drop redundant variables as needed. The salary dataset will also be cleaned and merged with the player statistics dataset to determine how many observations will be present.

Additionally, we must check for duplicate players on different teams throughout each season to account for trades. These cases will need to be combined to get the players complete statistics for the season. This should be as simple as aggregating numeric variables and leaving others along.

In [19]:
player_df = pd.read_csv("data/player_season.csv")
salary_full = pd.read_csv("data/player_salary.csv")

# updating the max seen columns to help see the data
pd.set_option("max_columns", player_df.shape[1])

player_df.head()
salary_full.head()

player_df.shape
salary_full.shape

player_df.isnull().sum()

,position,team,id,fullName,link,firstName,lastName,primaryNumber,birthDate,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,rookie,shootsCatches,season_type,season,timeOnIce,assists,goals,pim,shots,games,hits,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,penaltyMinutes,faceOffPct,shotPct,gameWinningGoals,overTimeGoals,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,plusMinus,points,shifts,timeOnIcePerGame,evenTimeOnIcePerGame,shortHandedTimeOnIcePerGame,powerPlayTimeOnIcePerGame,goalsInFirstPeriod,goalsInSecondPeriod,goalsInThirdPeriod,emptyNetGoals,shootOutGoals,shootOutShots,goalsTrailingByOne,goalsTrailingByTwo,goalsWhenTied,goalsLeadingByOne,goalsLeadingByTwo,goalsLeadingByThreePlus,penaltyGoals,penaltyShots,currentAge,alternateCaptain,captain,rosterStatus,goalsInOvertime,goalsTrailingByThreePlus
0,C,New Jersey Devils,8470619,Brian Boyle,/api/v1/people/8470619,Brian,Boyle,9.0,1984-12-18,Hingham,MA,USA,USA,"6' 6""",245,False,False,L,R,20182019,980:48,6,18,38,123,73,145,7,9,192:05,676:00,38,51.05,14.63,4,0,1,2,112:43,50,-14,24,1255,13:26,09:15,01:32,02:37,4.0,6.0,8.0,0.0,1.0,2.0,4.0,3.0,7.0,1.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,R,New Jersey Devils,8471226,Drew Stafford,/api/v1/people/8471226,Drew,Stafford,18.0,1985-10-30,Milwaukee,WI,USA,USA,"6' 2""",215,False,False,R,R,20182019,699:08,8,5,18,77,57,50,1,2,50:26,646:27,18,41.02,6.49,0,0,0,0,02:15,19,-7,13,868,12:15,11:20,00:02,00:53,2.0,3.0,NaN,0.0,2.0,6.0,2.0,2.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,C,New Jersey Devils,8471233,Travis Zajac,/api/v1/people/8471233,Travis,Zajac,14.0,1985-05-13,Winnipeg,MB,CAN,CAN,"6' 2""",185,True,False,R,R,20182019,1541:40,27,19,20,120,80,66,5,12,211:31,1112:40,20,58.16,15.83,3,1,2,2,217:29,38,-25,46,1818,19:16,13:54,02:43,02:38,7.0,8.0,3.0,0.0,0.0,1.0,2.0,1.0,6.0,4.0,2.0,1.0,0.0,0.0,35.0,False,False,Y,1.0,3.0
3,D,New Jersey Devils,8472382,Andy Greene,/api/v1/people/8472382,Andy,Greene,4.0,1982-10-30,Trenton,MI,USA,USA,"5' 11""",190,True,False,L,R,20182019,1708:22,20,5,16,104,82,67,0,2,09:09,1363:18,16,0.00,4.81,0,0,0,4,335:55,208,-9,25,2027,20:50,16:37,04:05,00:06,1.0,1.0,3.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,38.0,False,False,Y,NaN,2.0
4,D,New Jersey Devils,8473468,Eric Gryba,/api/v1/people/8473468,Eric,Gryba,2.0,1988-04-14,Saskatoon,SK,CAN,CAN,"6' 4""",222,False,False,R,R,20182019,152:57,0,0,10,11,10,21,0,0,00:07,140:09,10,0.00,0.00,0,0,0,0,12:41,16,-1,0,194,15:17,14:00,01:16,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,player,position,age,base_salary,signing_bonus,perf_bonus,total_salary,total_cap_hit,adjusted_cap_hit,cap_pct,team,season
0,Aaron Dell,G,29,"$1,950,000","$250,000",-,"$2,200,000","$1,900,000","$1,900,000",2.37,san-jose-sharks,2018/2019
1,Aaron Dell,G,30,"$1,600,000",-,-,"$1,600,000","$1,900,000","$1,900,000",2.62,san-jose-sharks,2019/2020
2,Aaron Ekblad,D,22,"$4,000,000",-,-,"$7,000,000","$7,500,000","$7,500,000",9.76,florida-panthers,2018/2019
3,Aaron Ekblad,D,23,"$5,000,000","$4,000,000",-,"$9,000,000","$7,500,000","$7,500,000",9.38,florida-panthers,2019/2020
4,Adam Boqvist,D,19,"$832,500","$92,500","$850,000","$705,712","$894,167","$658,559",0.95,chicago-blackhawks,2019/2020


(1947, 67)

(1482, 12)

position                       0
team                           0
id                             0
fullName                       0
link                           0
                            ... 
alternateCaptain             242
captain                      242
rosterStatus                 242
goalsInOvertime             1677
goalsTrailingByThreePlus    1371
Length: 67, dtype: int64

In [20]:
# removing unneeded columns and cleaning the salary columns
def clean_money(x):
    return x.str.replace("$", '').str.replace(",", '').str.replace("-", "0")
money_cols = ['base_salary', 'signing_bonus', 'perf_bonus', 'total_salary', 'total_cap_hit']

salary_full[money_cols] = salary_full[money_cols].apply(clean_money).astype(int)
salary_df = (salary_full.
             query("total_cap_hit != '-'").
             assign(
                 total_salary = lambda x: x.base_salary + x.signing_bonus + x.perf_bonus,
                 season = lambda x: x.season.str.replace("/", '')
             ).
             filter(['player', 'total_salary', 'total_cap_hit', 'season'])
            )
# salary_df.isnull().sum()

salary_df.head()

<ipython-input-20-6913f472d86e>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


,player,total_salary,total_cap_hit,season
0,Aaron Dell,2200000,1900000,20182019
1,Aaron Dell,1600000,1900000,20192020
2,Aaron Ekblad,4000000,7500000,20182019
3,Aaron Ekblad,9000000,7500000,20192020
4,Adam Boqvist,1775000,894167,20192020


In [21]:
pd.set_option("max_info_columns", player_df.shape[1])
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 67 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   position                     1947 non-null   object 
 1   team                         1947 non-null   object 
 2   id                           1947 non-null   int64  
 3   fullName                     1947 non-null   object 
 4   link                         1947 non-null   object 
 5   firstName                    1947 non-null   object 
 6   lastName                     1947 non-null   object 
 7   primaryNumber                1944 non-null   float64
 8   birthDate                    1947 non-null   object 
 9   birthCity                    1947 non-null   object 
 10  birthStateProvince           1393 non-null   object 
 11  birthCountry                 1947 non-null   object 
 12  nationality                  1947 non-null   object 
 13  height            

In [22]:
# player_df.groupby(by = ['fullName', 'season']).sum()
player_df.head()

,position,team,id,fullName,link,firstName,lastName,primaryNumber,birthDate,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,rookie,shootsCatches,season_type,season,timeOnIce,assists,goals,pim,shots,games,hits,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,penaltyMinutes,faceOffPct,shotPct,gameWinningGoals,overTimeGoals,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,plusMinus,points,shifts,timeOnIcePerGame,evenTimeOnIcePerGame,shortHandedTimeOnIcePerGame,powerPlayTimeOnIcePerGame,goalsInFirstPeriod,goalsInSecondPeriod,goalsInThirdPeriod,emptyNetGoals,shootOutGoals,shootOutShots,goalsTrailingByOne,goalsTrailingByTwo,goalsWhenTied,goalsLeadingByOne,goalsLeadingByTwo,goalsLeadingByThreePlus,penaltyGoals,penaltyShots,currentAge,alternateCaptain,captain,rosterStatus,goalsInOvertime,goalsTrailingByThreePlus
0,C,New Jersey Devils,8470619,Brian Boyle,/api/v1/people/8470619,Brian,Boyle,9.0,1984-12-18,Hingham,MA,USA,USA,"6' 6""",245,False,False,L,R,20182019,980:48,6,18,38,123,73,145,7,9,192:05,676:00,38,51.05,14.63,4,0,1,2,112:43,50,-14,24,1255,13:26,09:15,01:32,02:37,4.0,6.0,8.0,0.0,1.0,2.0,4.0,3.0,7.0,1.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,R,New Jersey Devils,8471226,Drew Stafford,/api/v1/people/8471226,Drew,Stafford,18.0,1985-10-30,Milwaukee,WI,USA,USA,"6' 2""",215,False,False,R,R,20182019,699:08,8,5,18,77,57,50,1,2,50:26,646:27,18,41.02,6.49,0,0,0,0,02:15,19,-7,13,868,12:15,11:20,00:02,00:53,2.0,3.0,NaN,0.0,2.0,6.0,2.0,2.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,C,New Jersey Devils,8471233,Travis Zajac,/api/v1/people/8471233,Travis,Zajac,14.0,1985-05-13,Winnipeg,MB,CAN,CAN,"6' 2""",185,True,False,R,R,20182019,1541:40,27,19,20,120,80,66,5,12,211:31,1112:40,20,58.16,15.83,3,1,2,2,217:29,38,-25,46,1818,19:16,13:54,02:43,02:38,7.0,8.0,3.0,0.0,0.0,1.0,2.0,1.0,6.0,4.0,2.0,1.0,0.0,0.0,35.0,False,False,Y,1.0,3.0
3,D,New Jersey Devils,8472382,Andy Greene,/api/v1/people/8472382,Andy,Greene,4.0,1982-10-30,Trenton,MI,USA,USA,"5' 11""",190,True,False,L,R,20182019,1708:22,20,5,16,104,82,67,0,2,09:09,1363:18,16,0.00,4.81,0,0,0,4,335:55,208,-9,25,2027,20:50,16:37,04:05,00:06,1.0,1.0,3.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,38.0,False,False,Y,NaN,2.0
4,D,New Jersey Devils,8473468,Eric Gryba,/api/v1/people/8473468,Eric,Gryba,2.0,1988-04-14,Saskatoon,SK,CAN,CAN,"6' 4""",222,False,False,R,R,20182019,152:57,0,0,10,11,10,21,0,0,00:07,140:09,10,0.00,0.00,0,0,0,0,12:41,16,-1,0,194,15:17,14:00,01:16,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# converting the time on ice columns to just minutes, removing the seconds after the : since
# those can be considered negligible
time_on_ice_cols = ['timeOnIce', 'powerPlayTimeOnIce', 'evenTimeOnIce', 'shortHandedTimeOnIce', 
                    'timeOnIcePerGame', 'evenTimeOnIcePerGame', 'shortHandedTimeOnIcePerGame', 
                    'powerPlayTimeOnIcePerGame']
# this isn't working quite right...
# player_df[time_on_ice_cols] = 
player_df[time_on_ice_cols].apply(lambda x: x.str.split(":", 1)[0]).astype(int)
# player_df['timeOnIce'].apply(lambda x: x.split(":", 1)[0]).astype(int)

,timeOnIce,powerPlayTimeOnIce,evenTimeOnIce,shortHandedTimeOnIce,timeOnIcePerGame,evenTimeOnIcePerGame,shortHandedTimeOnIcePerGame,powerPlayTimeOnIcePerGame
0,980,192,676,112,13,9,1,2
1,48,5,0,43,26,15,32,37


In [24]:
player_df['season'] = player_df['season'].astype(str)

# converting height to inches
r = re.compile(r"([0-9]+)' ([0-9]*\.?[0-9]+)\"")
def get_inches(height):
    m = r.match(height)
    if m == None:
        return float('NaN')
    else:
        return int(m.group(1))*12 + float(m.group(2))
    
player_df['height'] = player_df['height'].apply(get_inches)

In [120]:
# dropping unnecessary columns
player_df.drop(columns = ['team', 'birthDate', 'id', 'link', 'firstName', "lastName", 
                         'birthCity', 'birthStateProvince', 'birthCountry',
                         'active', 'rosterStatus', 'season_type'])

# player_df.groupby(by = ['fullName', 'season']).sum()

id  primaryNumber  weight  active  rookie  \
fullName          season                                                     
A.J. Greer        20182019  8478421           36.0     210       1       0   
Aaron Ekblad      20182019  8477932            5.0     215       1       0   
                  20192020  8477932            5.0     215       1       0   
Aaron Ness        20192020  8474604           42.0     184       1       0   
Adam Boqvist      20192020  8480871           27.0     179       1       0   
...                             ...            ...     ...     ...     ...   
Zack Smith        20192020  8474250           15.0     208       1       0   
Zdeno Chara       20182019  8465009           33.0     250       1       0   
                  20192020  8465009           33.0     250       1       0   
Zemgus Girgensons 20182019  8476878           28.0     213       1       0   
                  20192020  8476878           28.0     213       1       0   

                            timeOnIce  assists  goals  pim  shots  games  \
fullName          season                                                   
A.J. Greer        20182019        0.0        1      1   14      9     15   
Aaron Ekblad      20182019        0.0       24     13   47    180     82   
                  20192020        0.0       36      5   26    147     67   
Aaron Ness        20192020        0.0        1      0    0     11     24   
Adam Boqvist      20192020        0.0        9      4    6     50     41   
...                               ...      ...    ...  ...    ...    ...   
Zack Smith        20192020        0.0        7      4   29     43     50   
Zdeno Chara       20182019        0.0        9      5   57     99     62   
                  20192020        0.0        9      5   60     86     68   
Zemgus Girgensons 20182019        0.0       13      5   17     78     72   
                  20192020        0.0        7     12   10     85     69   

                            hits  powerPlayGoals  powerPlayPoints  \
fullName          season                                            
A.J. Greer        20182019    20               0                0   
Aaron Ekblad      20182019   114               5               10   
                  20192020    34               0                6   
Aaron Ness        20192020    17               0                0   
Adam Boqvist      20192020    33               0                2   
...                          ...             ...              ...   
Zack Smith        20192020   112               0                0   
Zdeno Chara       20182019    81               1                3   
                  20192020    78               0                0   
Zemgus Girgensons 20182019   144               0                0   
                  20192020   110               0                0   

                            powerPlayTimeOnIce  evenTimeOnIce  penaltyMinutes  \
fullName          season                                                        
A.J. Greer        20182019                 0.0            0.0              14   
Aaron Ekblad      20182019                 0.0            0.0              47   
                  20192020                 0.0            0.0              26   
Aaron Ness        20192020                 0.0            0.0               0   
Adam Boqvist      20192020                 0.0            0.0               6   
...                                        ...            ...             ...   
Zack Smith        20192020                 0.0            0.0              29   
Zdeno Chara       20182019                 0.0            0.0              57   
                  20192020                 0.0            0.0              60   
Zemgus Girgensons 20182019                 0.0            0.0              17   
                  20192020                 0.0            0.0              10   

                            faceOffPct  shotPct  gameWinningGoals  \
fullName

In [121]:
player_df.query('fullName == "Par Lindholm"')

,position,team,id,fullName,link,firstName,lastName,primaryNumber,birthDate,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,rookie,shootsCatches,season_type,season,timeOnIce,assists,goals,pim,shots,games,hits,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,penaltyMinutes,faceOffPct,shotPct,gameWinningGoals,overTimeGoals,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,plusMinus,points,shifts,timeOnIcePerGame,evenTimeOnIcePerGame,shortHandedTimeOnIcePerGame,powerPlayTimeOnIcePerGame,goalsInFirstPeriod,goalsInSecondPeriod,goalsInThirdPeriod,emptyNetGoals,shootOutGoals,shootOutShots,goalsTrailingByOne,goalsTrailingByTwo,goalsWhenTied,goalsLeadingByOne,goalsLeadingByTwo,goalsLeadingByThreePlus,penaltyGoals,penaltyShots,currentAge,alternateCaptain,captain,rosterStatus,goalsInOvertime,goalsTrailingByThreePlus
326,C,Toronto Maple Leafs,8480944,Par Lindholm,/api/v1/people/8480944,Par,Lindholm,26.0,1991-10-05,Kusmark,NaN,SWE,SWE,"6' 0""",181,False,False,L,R,20182019,NaN,12,1,18,65,65,58,0,0,NaN,NaN,18,50.12,1.54,0,0,0,0,NaN,55,5,13,1136,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
936,C,Winnipeg Jets,8480944,Par Lindholm,/api/v1/people/8480944,Par,Lindholm,26.0,1991-10-05,Kusmark,NaN,SWE,SWE,"6' 0""",181,False,False,L,R,20182019,NaN,12,1,18,65,65,58,0,0,NaN,NaN,18,50.12,1.54,0,0,0,0,NaN,55,5,13,1136,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1181,C,Boston Bruins,8480944,Par Lindholm,/api/v1/people/8480944,Par,Lindholm,26.0,1991-10-05,Kusmark,NaN,SWE,SWE,"6' 0""",181,False,False,L,R,20192020,NaN,3,3,4,35,40,23,0,0,NaN,NaN,4,45.83,8.60,1,0,0,0,NaN,26,6,6,632,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
player_df.select_dtypes("object")

,position,team,fullName,link,firstName,lastName,birthDate,birthCity,birthStateProvince,birthCountry,nationality,height,shootsCatches,season_type,alternateCaptain,captain,rosterStatus
0,C,New Jersey Devils,Brian Boyle,/api/v1/people/8470619,Brian,Boyle,1984-12-18,Hingham,MA,USA,USA,"6' 6""",L,R,NaN,NaN,NaN
1,R,New Jersey Devils,Drew Stafford,/api/v1/people/8471226,Drew,Stafford,1985-10-30,Milwaukee,WI,USA,USA,"6' 2""",R,R,NaN,NaN,NaN
2,C,New Jersey Devils,Travis Zajac,/api/v1/people/8471233,Travis,Zajac,1985-05-13,Winnipeg,MB,CAN,CAN,"6' 2""",R,R,False,False,Y
3,D,New Jersey Devils,Andy Greene,/api/v1/people/8472382,Andy,Greene,1982-10-30,Trenton,MI,USA,USA,"5' 11""",L,R,False,False,Y
4,D,New Jersey Devils,Eric Gryba,/api/v1/people/8473468,Eric,Gryba,1988-04-14,Saskatoon,SK,CAN,CAN,"6' 4""",R,R,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1942,C,Vegas Golden Knights,Nicolas Roy,/api/v1/people/8478462,Nicolas,Roy,1997-02-05,Amos,QC,CAN,CAN,"6' 4""",R,R,False,False,Y
1943,C,Vegas Golden Knights,Gage Quinney,/api/v1/people/8479748,Gage,Quinney,1995-07-29,Las Vegas,NV,USA,USA,"5' 11""",L,R,False,False,N
1944,D,Vegas Golden Knights,Nicolas Hague,/api/v1/people/8479980,Nicolas,Hague,1998-12-05,Kitchener,ON,CAN,CAN,"6' 6""",L,R,False,False,Y
1945,C,Vegas Golden Knights,Cody Glass,/api/v1/people/8479996,Cody,Glass,1999-04-01,Winnipeg,MB,CAN,CAN,"6' 3""",R,R,False,False,N
